## Matching industry with NAICS code and salary

In [1]:
import numpy as np
import pandas as pd

### 1. Matching NAICS code with average salary

We download OEWS wage research estimates by state and industry data from Bureau of labor statistics. In this step, what we do is grouping by NAICS sector-level industry and calculating the average annual industry salary across the US.

#### (1) Read data
This file is too big that can not be uploaded into github.
Please download this file from this link: https://docs.google.com/spreadsheets/d/1_H4-EzobjeBscFc5EhBwsyXd89lR4awd/edit?usp=sharing&ouid=107274150749596815334&rtpof=true&sd=true

In [4]:
# read data
salary = pd.read_excel('/Users/ning/Desktop/Uchicago/Winter 2022/30122_Application/final-project/oes_research_2020_allsectors.xlsx', sheet_name = 'state_industry_M2020')

In [5]:
salary.head()

,AREA,AREA_TITLE,NAICS,NAICS_TITLE,I_GROUP,OCC_CODE,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,1,Alabama,11,"Agriculture, Forestry, Fishing and Hunting",sector,00-0000,All Occupations,total,5980,4.6,...,17.18,23.05,30.05,20200,26560,35740,47940,62500,NaN,NaN
1,1,Alabama,11,"Agriculture, Forestry, Fishing and Hunting",sector,11-0000,Management Occupations,major,90,15.1,...,60.68,79.92,97.44,56890,81000,126210,166240,202680,NaN,NaN
2,1,Alabama,11,"Agriculture, Forestry, Fishing and Hunting",sector,11-1021,General and Operations Managers,detailed,50,13.6,...,63.37,78.1,93.67,69360,80950,131800,162450,194820,NaN,NaN
3,1,Alabama,11,"Agriculture, Forestry, Fishing and Hunting",sector,13-0000,Business and Financial Operations Occupations,major,100,35.7,...,37.78,54.83,63.7,52200,58410,78580,114040,132490,NaN,NaN
4,1,Alabama,11,"Agriculture, Forestry, Fishing and Hunting",sector,13-1020,Buyers and Purchasing Agents,detailed,60,41.9,...,30.25,41.31,69.77,52560,56520,62910,85930,145120,NaN,NaN


#### (2) Clean data

In [6]:
# drop "A_MEAN" rows including "#" and "*"
useless_value = ['#', "*"]
salary = salary[salary.A_MEAN.isin(useless_value) == False]

# covert "A_MEAN" into int
salary['A_MEAN'] = pd.to_numeric(salary['A_MEAN'])

# extract necessary columns
salary = salary[['NAICS', 'NAICS_TITLE', 'I_GROUP', 'A_MEAN']]

# drop duplicated rows
salary = salary.drop_duplicates()

# select sector-level NAICS
salary = salary[salary['I_GROUP'] == 'sector']
salary = salary[['NAICS', 'NAICS_TITLE', 'A_MEAN']]

# calculate the mean value of every NAICS sector
map_code_salary = salary.groupby(['NAICS', 'NAICS_TITLE']).mean().round(3)

# preview completed matching table
map_code_salary

,,A_MEAN
NAICS,NAICS_TITLE,
11,"Agriculture, Forestry, Fishing and Hunting",47880.873
21,"Mining, Quarrying, and Oil and Gas Extraction",69910.580
22,Utilities,84207.831
23,Construction,64648.972
31-33,Manufacturing,69174.562
42,Wholesale Trade,68623.359
44-45,Retail Trade,56013.266
48-49,Transportation and Warehousing,65503.789
51,Information,73855.595


#### (3) Save data
This table has been save as "industry_code_with_salary.csv" and uploaded to github. You can skip this part and directly read this csv file for the next step.

In [46]:
# write csv
map_code_salary.to_csv("industry_code_with_salary.csv",index = True)

### 2. Matching industry with NAICS code and average salary

We've scrapped industry information from EBSCO. But the industry NAICS code we scraped is 6-digit more detailed industry classfication. To match current company with salary, we need to transfer scrapped NAICS code into 2-digit sector-level NAICS code from Bureau of labor statistics and then match it with salary.

#### (1) Read data

In [7]:
# set your school name
school_name = 'Harvard University'

In [8]:
# please download this csv file from github
map_salary = pd.read_csv('./industry_code_with_salary.csv')

In [10]:
# read your school profile file
individual_profile = pd.read_csv('/Users/ning/Desktop/Uchicago/Winter 2022/30122_Application/final-project/school_industry/rank02/Harvard University_with_industry_code0.csv')

#### (2) Clean data

In [11]:
# extract NAICS code from school profile
individual_profile['current_NAICS'] = individual_profile.current_company_industry.str.extract(r'(\d\d)',expand=False)
individual_profile['previous_NAICS'] = individual_profile.previous_company_industry.str.extract(r'(\d\d)',expand=False)

# deal with "31-33", "44-45", "48-49" NAICS code
individual_profile['current_NAICS'] = individual_profile['current_NAICS'].replace(['31','32','33'], '31-33')
individual_profile['current_NAICS'] = individual_profile['current_NAICS'].replace(['44', '45'], '44-45')
individual_profile['current_NAICS'] = individual_profile['current_NAICS'].replace(['48', '49'], '48-49')

individual_profile['previous_NAICS'] = individual_profile['previous_NAICS'].replace(['31','32','33'], '31-33')
individual_profile['previous_NAICS'] = individual_profile['previous_NAICS'].replace(['44', '45'], '44-45')
individual_profile['previous_NAICS'] = individual_profile['previous_NAICS'].replace(['48', '49'], '48-49')

#### (3) Match data

In [132]:
# map current NAICS with title and average salary
individual_profile = pd.merge(individual_profile, map_salary, left_on='current_NAICS', right_on='NAICS', how='left')\
.rename(columns={'NAICS_TITLE': 'current_NAICS_title', 'A_MEAN': 'current_avg_salary'}).drop(columns=['NAICS'])

# map previous NAICS with title
individual_profile = pd.merge(individual_profile, map_salary, left_on='previous_NAICS', right_on='NAICS', how='left')\
.rename(columns={'NAICS_TITLE': 'previous_NAICS_title'}).drop(columns=['NAICS', 'A_MEAN'])

In [138]:
individual_profile.head()

,Unnamed: 0,Unnamed: 0.1,current_location,current_work,previous_work,latest_Education,major,search_school,current_company,previous_company,current_company_industry,previous_company_industry,current_NAICS,previous_NAICS,current_NAICS_title,current_avg_salary,previous_NAICS_title
0,0,0,"San Francisco, California, United States",Chief Operating Officer at Hypersphere Ventures,"Investor, Owner, & Operator at Force Factor",Harvard University,Biology,Harvard University,Hypersphere Ventures,Force Factor,NaN,424210 Vitamins merchant wholesalers,NaN,42,NaN,NaN,Wholesale Trade
1,1,1,"Palatine, Illinois, United States",Chief Technology Officer at Videojet Technologies,Chief Technology Officer/Vice President of R&D...,Collège de France,Polymer Chemistry and Physics,Harvard University,Videojet Technologies,"Videojet Technologies Inc., a Danaher Company",335999 UPS (uninterruptible power supplies) ma...,NaN,31-33,NaN,Manufacturing,69174.562,NaN
2,2,2,"Chicago, Illinois, United States",Co-Founder & CTO at Science On Call: Tech Supp...,Director of Technology & Marketing at Standard...,Northwestern University - Kellogg School of Ma...,"Marketing, Entrepreneurship, Strategy, Interna...",Harvard University,Science On Call: Tech Support for Restaurants,Standard Market,NaN,"519110 Wire services, news",NaN,51,NaN,NaN,Information
3,3,3,Stockholm Metropolitan Area,CEO & Co-Founder at Learnifier,Co-Founder & CEO at Dabox,Harvard University,Systems Science & Business Management,Harvard University,Learnifier,Dabox,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,"New York, New York, United States",Head of Digital Experience at MassMutual,"Board Member, MassMutual Foundation at MassMutual",+ 2 more,Social Studies,Harvard University,MassMutual,MassMutual,"531210 Selling real estate for others (i.e., a...","531210 Selling real estate for others (i.e., a...",53,53,Real Estate and Rental and Leasing,60268.137,Real Estate and Rental and Leasing


#### (4) Write data

In [136]:
# write csv
filename = str(school_name)+"with_salary&NAICS.csv"
individual_profile.to_csv(filename,index = True)